In [1]:
%use kandy
import java.io.File

In [2]:
data class Noun(
    val word: String,
    val ipm: Double,
    val juillandCoeff: Double,
)

In [3]:
val nouns = File("freqrnc2011.csv")
    .readLines()
    .drop(1)
    .mapNotNull {line: String ->
        val (word, partSpeech, ipm, _, juillandCoeff) = line.split(",")
        if (partSpeech == "s") 
            Noun(word, ipm.toDouble(), juillandCoeff.toDouble())
        else null
    }
    .filter { noun -> noun.word.length >= 3}
    
nouns.take(10)
    .joinToString("\n")

Noun(word=абажур, ipm=4.9, juillandCoeff=90.0)
Noun(word=аббат, ipm=1.6, juillandCoeff=79.0)
Noun(word=аббатство, ipm=0.9, juillandCoeff=77.0)
Noun(word=аббревиатура, ipm=3.5, juillandCoeff=91.0)
Noun(word=абвер, ipm=2.7, juillandCoeff=38.0)
Noun(word=аберрация, ipm=0.7, juillandCoeff=79.0)
Noun(word=абзац, ipm=10.2, juillandCoeff=87.0)
Noun(word=абитуриент, ipm=4.0, juillandCoeff=88.0)
Noun(word=абонемент, ipm=1.8, juillandCoeff=87.0)
Noun(word=абонент, ipm=10.6, juillandCoeff=83.0)

In [4]:
val nounsSortedByDifficulty = nouns.sortedByDescending {noun -> noun.juillandCoeff}
nounsSortedByDifficulty.take(10).joinToString("\n")

Noun(word=конец, ipm=634.7, juillandCoeff=98.0)
Noun(word=место, ipm=926.6, juillandCoeff=98.0)
Noun(word=час, ipm=468.1, juillandCoeff=98.0)
Noun(word=вещь, ipm=242.5, juillandCoeff=97.0)
Noun(word=внимание, ipm=286.0, juillandCoeff=97.0)
Noun(word=время, ipm=2015.7, juillandCoeff=97.0)
Noun(word=выход, ipm=147.0, juillandCoeff=97.0)
Noun(word=дело, ipm=1412.1, juillandCoeff=97.0)
Noun(word=день, ipm=1258.4, juillandCoeff=97.0)
Noun(word=достоинство, ipm=66.8, juillandCoeff=97.0)

In [5]:
import kotlin.random.Random
val rnd = Random(23)
val easyNouns = nounsSortedByDifficulty.slice(0..2000).shuffled(rnd)
val medNouns = nounsSortedByDifficulty.slice(2000..5000).shuffled(rnd)
// easyNouns.take(10).joinToString("\n")
medNouns.take(10).joinToString("\n")

Noun(word=королевство, ipm=7.4, juillandCoeff=90.0)
Noun(word=идеалист, ipm=3.7, juillandCoeff=89.0)
Noun(word=полслова, ipm=5.2, juillandCoeff=92.0)
Noun(word=перечисление, ipm=6.9, juillandCoeff=90.0)
Noun(word=авантюра, ipm=5.5, juillandCoeff=91.0)
Noun(word=вестибюль, ipm=10.6, juillandCoeff=91.0)
Noun(word=экипаж, ipm=34.8, juillandCoeff=90.0)
Noun(word=починка, ipm=1.9, juillandCoeff=90.0)
Noun(word=листовка, ipm=8.7, juillandCoeff=91.0)
Noun(word=струна, ipm=13.8, juillandCoeff=92.0)

In [6]:
// flashcards information -- rows/columns of thr cards table per on a single page and the number of pages
val rows = 16
val columns = 5
val pages = 2 // number of pages on which the cards are printed
val excess = 20 // you will have to throw away some words while filtering words by hand 
val totalWords = rows * columns * pages + excess
println("In total $totalWords words will be played in the game")

In total 160 words will be played in the game


In [7]:
val sliceBegin = 0 // to use the single seeded vocabulary for many words without intersections
val wordlist = easyNouns.slice(sliceBegin..(sliceBegin + totalWords))
val wordlistLatexFile = File("./src-template/wordlist.tex")
wordlistLatexFile.writeText(
    wordlist.map { noun -> noun.word + " \\newpage" }.joinToString("\n")
)

In [8]:
// a line for making a pdf of needed size
// insert it manually -- you'll have to tweak the font anyway
val a4width = 29.7 // cm
val a4height = 21.0 // cm
val column_width = a4width / columns
val row_height = a4height / rows
fun Double.format(digits: Int) = "%.${digits}f".format(this)
println("\\usepackage[paperheight=${row_height.format(4)}cm,paperwidth=${column_width.format(4)}cm, top=0.4cm,left=0pt, right=0pt]{geometry}")

\usepackage[paperheight=1.3125cm,paperwidth=5.9400cm, top=0.4cm,left=0pt, right=0pt]{geometry}


In [9]:
import java.util.concurrent.TimeUnit
fun String.runCommand(
    workingDir: File = File("."),
    timeoutAmount: Long = 2,
    timeoutUnit: TimeUnit = TimeUnit.SECONDS
): String? = runCatching {
    ProcessBuilder("\\s".toRegex().split(this))
        .directory(workingDir)
        .redirectOutput(ProcessBuilder.Redirect.PIPE)
        .redirectError(ProcessBuilder.Redirect.PIPE)
        .start()
        .also { it.waitFor(timeoutAmount, timeoutUnit) }
        .inputStream.bufferedReader().readText()
}.onFailure { it.printStackTrace() }.getOrNull() // copypasted from stack overflow


File("./out").mkdirs() // pdflatex is stupid and cannot create an empty directory
"pdflatex -file-line-error -interaction=nonstopmode -synctex=0 -output-format=pdf -output-directory=../out main.tex".runCommand(File("./src-template/"))

// the end result should be printed on one side of page each, (rows \times columns) file pages per a physical page

This is pdfTeX, Version 3.141592653-2.6-1.40.23 (TeX Live 2021/W32TeX) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./main.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-02-24>
(c:/texlive/2021/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(c:/texlive/2021/texmf-dist/tex/latex/base/size11.clo))
(c:/texlive/2021/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(c:/texlive/2021/texmf-dist/tex/latex/amsmath/amstext.sty
(c:/texlive/2021/texmf-dist/tex/latex/amsmath/amsgen.sty))
(c:/texlive/2021/texmf-dist/tex/latex/amsmath/amsbsy.sty)
(c:/texlive/2021/texmf-dist/tex/latex/amsmath/amsopn.sty))
(c:/texlive/2021/texmf-dist/tex/latex/carlisle/scalefnt.sty)
(c:/texlive/2021/texmf-dist/tex/latex/fontsize/fontsize.sty
(c:/texlive/2021/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(c:/texlive/2021/texmf-dist/tex/generic/xkeyval/xkeyval

In [17]:
// various statistical information 
println(nounsSortedByDifficulty
    .filter { noun -> noun.word.length > 20 }
    .map { it.word }
    .take(10)
    .joinToString("\n")
)

plot {
    histogram(nouns.map {noun -> noun.word.length }) {
        x.axis.name = "word length"
    }
}

достопримечательность
самосовершенствование
самоусовершенствование
конкурентоспособность
высокопревосходительство


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EZMmwc"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"word length",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[3.15,4.199999999999999,5.25,6.300000000000001,7.35,8.4,9.450000000000001,10.5,11.55,12.600000000000001,13.65,14.700000000000001,15.75,16.8,17.85,18.9,19.950000000000003,21.0,22.05,23.1],
"count":[385.0,1221.0,2380.0,2847.0,3230.0,2988.0,2493.0,3400.0,978.0,781.0,533.0,320.0,210.0,111.0,76.0,33.0,12.0,3.0,1.0,1.0]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("EZMmwc");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 word length